In [86]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import project_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")


In [87]:
kappa = Eigen.Scalar("kappa")
v = Eigen.Scalar("v")
q = Eigen.Vector("q", 12)
q_a = Matrix(q[3:12])

Cl = Gen.Closure(kappa, v, q)

q_a

Matrix([
[ q(3)],
[ q(4)],
[ q(5)],
[ q(6)],
[ q(7)],
[ q(8)],
[ q(9)],
[q(10)],
[q(11)]])

In [88]:
a = [Matrix(q_a[0:3]), Matrix(q_a[3:6]), Matrix(q_a[6:9])]

In [89]:
E = 0
for i in range(3):
    E+=(a[i].dot(a[i]) - 1)**2
for i in range(3):
    for j in range(3):
        if i!=j:
            E+=(a[i].dot(a[j]))**2

E*=kappa * v

E

Matrix([[kappa*v*(2*(q(10)*q(4) + q(11)*q(5) + q(3)*q(9))**2 + 2*(q(10)*q(7) + q(11)*q(8) + q(6)*q(9))**2 + 2*(q(3)*q(6) + q(4)*q(7) + q(5)*q(8))**2 + (q(10)**2 + q(11)**2 + q(9)**2 - 1)**2 + (q(3)**2 + q(4)**2 + q(5)**2 - 1)**2 + (q(6)**2 + q(7)**2 + q(8)**2 - 1)**2)]])

In [90]:
dEdq = VecDiff(E,q_a)
dEdq

Matrix([
[     kappa*v*(4*q(3)*(q(3)**2 + q(4)**2 + q(5)**2 - 1) + 4*q(6)*(q(3)*q(6) + q(4)*q(7) + q(5)*q(8)) + 4*q(9)*(q(10)*q(4) + q(11)*q(5) + q(3)*q(9)))],
[    kappa*v*(4*q(10)*(q(10)*q(4) + q(11)*q(5) + q(3)*q(9)) + 4*q(4)*(q(3)**2 + q(4)**2 + q(5)**2 - 1) + 4*q(7)*(q(3)*q(6) + q(4)*q(7) + q(5)*q(8)))],
[    kappa*v*(4*q(11)*(q(10)*q(4) + q(11)*q(5) + q(3)*q(9)) + 4*q(5)*(q(3)**2 + q(4)**2 + q(5)**2 - 1) + 4*q(8)*(q(3)*q(6) + q(4)*q(7) + q(5)*q(8)))],
[     kappa*v*(4*q(3)*(q(3)*q(6) + q(4)*q(7) + q(5)*q(8)) + 4*q(6)*(q(6)**2 + q(7)**2 + q(8)**2 - 1) + 4*q(9)*(q(10)*q(7) + q(11)*q(8) + q(6)*q(9)))],
[    kappa*v*(4*q(10)*(q(10)*q(7) + q(11)*q(8) + q(6)*q(9)) + 4*q(4)*(q(3)*q(6) + q(4)*q(7) + q(5)*q(8)) + 4*q(7)*(q(6)**2 + q(7)**2 + q(8)**2 - 1))],
[    kappa*v*(4*q(11)*(q(10)*q(7) + q(11)*q(8) + q(6)*q(9)) + 4*q(5)*(q(3)*q(6) + q(4)*q(7) + q(5)*q(8)) + 4*q(8)*(q(6)**2 + q(7)**2 + q(8)**2 - 1))],
[ kappa*v*(4*q(3)*(q(10)*q(4) + q(11)*q(5) + q(3)*q(9)) + 4*q(6)*(q(10)*q(7) + q(11)*

In [91]:
ddEddq = VecDiff(dEdq, q_a)
ddEddq

Matrix([
[kappa*v*(12*q(3)**2 + 4*q(4)**2 + 4*q(5)**2 + 4*q(6)**2 + 4*q(9)**2 - 4),                        kappa*v*(4*q(10)*q(9) + 8*q(3)*q(4) + 4*q(6)*q(7)),                        kappa*v*(4*q(11)*q(9) + 8*q(3)*q(5) + 4*q(6)*q(8)),                        kappa*v*(8*q(3)*q(6) + 4*q(4)*q(7) + 4*q(5)*q(8)),                                                       4*kappa*q(4)*q(6)*v,                                                       4*kappa*q(5)*q(6)*v,                        kappa*v*(4*q(10)*q(4) + 4*q(11)*q(5) + 8*q(3)*q(9)),                                                        4*kappa*q(4)*q(9)*v,                                                        4*kappa*q(5)*q(9)*v],
[                      kappa*v*(4*q(10)*q(9) + 8*q(3)*q(4) + 4*q(6)*q(7)), kappa*v*(4*q(10)**2 + 4*q(3)**2 + 12*q(4)**2 + 4*q(5)**2 + 4*q(7)**2 - 4),                       kappa*v*(4*q(10)*q(11) + 8*q(4)*q(5) + 4*q(7)*q(8)),                                                      4*kappa*q(3)*q(7)*v,               

In [92]:
s = f'''// Affine body Orthogonal Potential

{Cl("E",E)}
{Cl("dEdq",dEdq)}
{Cl("ddEddq",ddEddq)}
'''
print(s)

f = open( project_dir() / 'src/uipc/backends/cuda/affine_body/constitutions/sym/ortho_potential.inl', 'w')
f.write(s)
f.close()


// > Squared Version
// > D := d*d 
// > DHat := dHat*dHat

template <typename T>
__host__ __device__ void E(T& R, const T& kappa, const T& v, const Eigen::Vector<T,12>& q)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
LaTeX expression:
//tex:$$R = \left[\begin{matrix}\kappa v \left(2 \left(q(10) q(4) + q(11) q(5) + q(3) q(9)\right)^{2} + 2 \left(q(10) q(7) + q(11) q(8) + q(6) q(9)\right)^{2} + 2 \left(q(3) q(6) + q(4) q(7) + q(5) q(8)\right)^{2} + \left(q(10)^{2} + q(11)^{2} + q(9)^{2} - 1\right)^{2} + \left(q(3)^{2} + q(4)^{2} + q(5)^{2} - 1\right)^{2} + \left(q(6)^{2} + q(7)^{2} + q(8)^{2} - 1\right)^{2}\right)\end{matrix}\right]$$

Symbol Name Mapping:
